In [96]:
import pandas as pd
import numpy as np
import copy

# Camiones y Camión

In [195]:
df_clientes = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="clientes", index_col=[0])
df_clientes

,Coord x,Coord y
Cliente,,
A,0.257571,1.803726
B,1.523313,2.102301
C,0.710910,2.629800
D,1.013940,2.149740
E,0.609480,0.666810
F,0.998640,1.451790
G,1.434510,0.018180
H,2.171880,0.764910
I,1.412280,0.863190


In [4]:
df_camiones = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="camiones")
df_camiones

,camion,carga_max,clientes_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


In [5]:
df_pedidos = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="pedidos")
df_pedidos

,Cliente,Pedidos #1,Pedidos #2,Pedidos #3,Pedidos #4,Pedidos #5,Pedidos #6
0,A,4,3,5,2,6,6
1,B,6,6,5,4,1,3
2,C,1,6,6,2,6,7
3,D,8,6,5,2,8,2
4,E,4,1,2,4,2,3
5,F,3,1,6,8,6,5
6,G,3,2,3,7,4,3
7,H,5,8,5,2,4,8
8,I,5,2,3,4,2,6
9,J,1,8,6,8,1,2


In [192]:
df_distancias = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="distancias", index_col=[0])
df_distancias

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
Cliente,,,,,,,,,,,,,,,
A,0.000000,1.300481,0.942292,0.831757,1.190134,0.820391,2.138542,2.178008,1.489282,2.169046,2.899109,2.047357,1.491853,2.250309,1.767958
B,1.300481,0.000000,0.968635,0.511577,1.701683,0.835731,2.086012,1.486356,1.244076,1.120697,2.068003,0.747466,0.693247,1.833682,2.088735
C,0.942292,0.968635,0.000000,0.567701,1.965609,1.212640,2.710010,2.369018,1.900745,2.085834,3.008383,1.563518,1.554526,2.621602,2.501215
D,0.831757,0.511577,0.567701,0.000000,1.537098,0.698118,2.172654,1.805154,1.346806,1.569917,2.459347,1.237830,0.994192,2.055756,2.026486
E,1.190134,1.701683,1.965609,1.537098,0.000000,0.876150,1.049474,1.565477,0.826470,1.917289,2.253598,2.315795,1.334625,1.383229,0.579504
F,0.820391,0.835731,1.212640,0.698118,0.876150,0.000000,1.498406,1.359521,0.719408,1.389120,2.078832,1.510223,0.704151,1.478470,1.330847
G,2.138542,2.086012,2.710010,2.172654,1.049474,1.498406,0.000000,1.049438,0.845302,1.667396,1.510842,2.429644,1.439609,0.594991,0.613642
H,2.178008,1.486356,2.369018,1.805154,1.565477,1.359521,1.049438,0.000000,0.765932,0.658239,0.727953,1.546591,0.816165,0.499332,1.482584
I,1.489282,1.244076,1.900745,1.346806,0.826470,0.719408,0.845302,0.765932,0.000000,1.106023,1.485664,1.667471,0.639137,0.764083,0.934229


In [193]:
df_distancias.loc["A", "B"]

1.30048061930542

In [198]:
print(df_clientes.loc["A"])
print(df_clientes.loc["B"])

Coord x    0.257571
Coord y    1.803726
Name: A, dtype: float64
Coord x    1.523313
Coord y    2.102301
Name: B, dtype: float64


In [215]:
for cod in list(df_clientes.index)[1:]:
    print(round(((df_clientes.loc["A"]["Coord x"] - df_clientes.loc[cod]["Coord x"])**2 + (df_clientes.loc["A"]["Coord y"] - df_clientes.loc[cod]["Coord y"])**2)**(1/2), 1))

1.3
0.9
0.8
1.2
0.8
2.1
2.2
1.5
2.2
2.9
2.0
1.5
2.3
1.8


In [202]:
((df_clientes.loc["A"]["Coord x"] - df_clientes.loc["B"]["Coord x"])**2 + (df_clientes.loc["A"]["Coord y"] - df_clientes.loc["B"]["Coord y"])**2)**(1/2)

1.3004806193054166

In [262]:
class Camion(object):
    
    def __init__(self, ix, carga_max, pedidos_max, dist_max):
        self.ix = ix
        self.carga_max = carga_max
        self.pedidos_max = pedidos_max
        self.dist_max = dist_max
        self.pedidos_asignados = {}
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
    def __repr__(self):
        return f'Camión {self.ix}'
    
    def __str__(self):
        return f'Camión {self.ix}'
    
    def _check_pedido_carga(self, pedido):
        return (self.carga_total + pedido.carga) <= self.carga_max
    
    def _check_pedido_cantidad(self, pedido):
        return self.cantidad_pedidos < self.pedidos_max
    
    def _check_pedido_distancia(self, pedido):
        dist_check = [pedido.distancia(other) <= self.dist_max for other in self.pedidos_asignados.values()]
        return all(dist_check)
    
    def check_pedido(self, pedido):
        return not pedido.asignado and self._check_pedido_carga(pedido) and self._check_pedido_cantidad(pedido) and self._check_pedido_distancia(pedido)
        # if pedido.asignado or self._check_pedido_carga(pedido) or self._check_pedido_cantidad(pedido):
        # # Falta distancias.
        #     return False
        # else:
        #     return True
    
    def add_pedido(self, pedido):
        if self.check_pedido(pedido):
            self.pedidos_asignados[pedido.ix] = pedido
            self.carga_total += pedido.carga
            self.cantidad_pedidos += 1
            pedido.asignado = True
            # Actualizar carga_total
            # Actualizar n_pedidos
    
    def remove_pedido(self, pedido_ix):
        self.pedidos_asignados.get(pedido_ix).asignado = False
        self.carga_total -= self.pedidos_asignados.get(pedido_ix).carga
        self.cantidad_pedidos -= 1
        self.pedidos_asignados.pop(pedido_ix)
        
    #def _set_carga_total(self):
    #    cargas = [ped.carga for ped in self.pedidos_asignados.values()]
    #    # Tendría que guardar en un atributo este dato?
    #    # Se tendría que actualizar cada vez que cargo un nuevo pedido?
    #    self.carga_total = sum(cargas)
    #    #return sum(cargas)
    
    def get_carga_total(self):
        return self.carga_total
    
    def get_costo(self):
        # Tendría que guardar en un atributo este dato?
        # Se tendría que actualizar cada vez que cargo un nuevo pedido?
         
        if self.carga_total == 0:
            costo = 5000
        elif self.carga_total <= 4:
            costo = 5600
        elif self.carga_total > 4 and self.carga_total < 6.5:
            costo = 1400*self.carga_total
        elif self.carga_total >= 6.5 and self.carga_total < 9.5:
            costo = 1200*self.carga_total
        else:
            costo = 1000*self.carga_total
                
        return costo 
    
    def get_costo_tn(self):
        return self.get_costo()/self.carga_total
    
    def count_pedidos(self):
        return self.cantidad_pedidos
    
    def reset_pedidos(self):
        self.pedidos_asignados.clear()
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
        
        
        
class Pedido(object):
    
    #def __init__(self, ix):
    def __init__(self, ix, x, y, carga):
        self.ix = ix
        self.x = x
        self.y = y
        self.carga = carga
        self.asignado = False
        
    def __repr__(self):
        return f'Pedido {self.ix}'
    
    def __str__(self):
        return f'Pedido {self.ix}'
    
    def get_ix(self):
        return self.ix
    
    def get_carga(self):
        return self.carga
    
    def distancia(self, other):
        dist = ((self.x - other.x)**2 + (self.y - other.y)**2)**(1/2)
        return round(dist, 1)
        

In [246]:
pedido = Pedido(ix="C", x=2, y=2, carga=11)

[pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()]

[False, False]

In [247]:
any([pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()])

False

In [250]:
not False and False

False

In [221]:
camion1.pedidos_asignados

{}

In [53]:
x = {}
x["A"] = Pedido("A")
x["B"] = Pedido("B")
print(x)
x.pop("B")
print(x)
x.clear()
print(x)

{'A': Pedido A, 'B': Pedido B}
{'A': Pedido A}
{}


In [238]:
camion1 = Camion(ix=1, carga_max=12, pedidos_max=3, dist_max=2)
camion2 = Camion(ix=2, carga_max=12, pedidos_max=3, dist_max=2)

camion1.add_pedido(Pedido(ix="A", x=1, y=1, carga=4))
camion1.add_pedido(Pedido(ix="B", x=3, y=3, carga=5))
camion2.add_pedido(Pedido(ix="C", x=1, y=1, carga=11))

print(camion1.count_pedidos())
print(camion2.count_pedidos())
print(camion1.get_carga_total())
print(camion2.get_carga_total())
print(camion1.get_costo())
print(camion2.get_costo())
print(camion1.get_costo_tn())
print(camion2.get_costo_tn())


1
1
5
11
7000
11000
1400.0
1000.0


In [263]:
camion1 = Camion(ix=1, carga_max=12, pedidos_max=3, dist_max=2)

camion1.add_pedido(Pedido(ix="A", x=1, y=1, carga=4))
print(camion1.get_carga_total())
camion1.add_pedido(Pedido(ix="B", x=1, y=1, carga=5))
print(camion1.get_carga_total())
camion1.add_pedido(Pedido(ix="C", x=1, y=1, carga=4))
print(camion1.get_carga_total())

camion1.pedidos_asignados

True
True
True
True
True
4
True
True
True
True
True
9
True
False
True
True
False
9


{'A': Pedido A, 'B': Pedido B}

In [261]:
x = 1
x -= 1
print(x)

0


## Pruebas con datos reales

In [14]:
camion_prueba = df_camiones.iloc[0,:]
camion_prueba

camion           1
carga_max       12
clientes_max     3
dist_max         2
Name: 0, dtype: int64

In [155]:
camion = Camion(ix=camion_prueba.camion,
                carga_max=camion_prueba.carga_max,
                pedidos_max=camion_prueba.clientes_max,
                dist_max=camion_prueba.dist_max)

In [76]:
pedido_prueba = df_pedidos.iloc[0,0:2]
pedido_prueba

cliente_prueba = df_clientes.iloc[0]
cliente_prueba

Cliente           A
Coord x    0.257571
Coord y     1.80373
Name: 0, dtype: object

In [156]:
pedido = Pedido(ix=pedido_prueba.Cliente,
                x=cliente_prueba["Coord x"],
                y=cliente_prueba["Coord y"],
                carga=pedido_prueba["Pedidos #1"])

In [157]:
camion.add_pedido(pedido)

Para controlar cómo copiar y guardar elementos en los camiones. Pensar qué pasaría si cargo muchos objetos en memoria.

In [137]:
# Pruebas de memoria para realizar cambios -> Ojo con modificar objetos.
camion1 = Camion("1", 2, 2, 2)
#camion2 = copy.copy(camion1)
camion2 = copy.deepcopy(camion1)
pedido1 = Pedido("A", 1, 1, 4)

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

#pedido.carga = 5
#camion.pedidos_asignados.get("A").carga = 5
camion2.pedidos_asignados.get("A").carga = 5

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

4
4
4
4
5
4


In [175]:
class A(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class B(A):
    
    def __init__(self):
        self.x = A.x

In [176]:
a = A(1,2)
b = B(a)

TypeError: __init__() takes 1 positional argument but 2 were given